## 1. 416 - Partition Equal Subset Sum
Given a non-empty array containing only positive integers, find if the array can be partitioned into two subsets such that the sum of elements in both subsets is equal.

**idea** package problem  
record every possible value of sum for every num in nums.

In [1]:
class Solution(object):
    def canPartition(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        targets = sum(nums)
        if targets%2 == 1: return False
        targets >>= 1
        
        lst = {0}
        # lst = [False]*(targets+1)
        for num in nums:
            lst |= {x+num for x in lst}
            if targets in lst: return True
        
        return False

# 2. 805 - Split Array With Same Average
In a given integer array A, we must move every element of A to either list B or list C. (B and C initially start empty.)  
Return true if and only if after such a move, it is possible that the average value of B is equal to the average value of C, and B and C are both non-empty.

**idea** A[i] = A[i] * n - total >> A's sum and average will be zero  
length will not affect the average of subarray

1. just like split array with same sum
2. record the shortest length of a kind of sum will be enough

In [2]:
class Solution(object):
    def splitArraySameAverage(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        total, n = sum(A), len(A)
        for i in range(len(A)):
            A[i] = A[i] * n - total

        possible = {A[0]:1}
        
        for a in A[1:]:
            for k in possible.keys():
                if k+a not in possible and possible[k] < n-1:
                    possible[k+a] = possible[k] + 1
            if 0 in possible: return True
        
        return False 

notes:  
we can add neg and pos separately to see if they add up to zero

In [4]:
class Solution(object):
    def splitArraySameAverage(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        n, total = len(A), sum(A)
        big, small = set(), set()
        average = total / float(n)
        
        for num in A[:-1]:
            if num == average:
                return True
            elif num > average:
                big |= {x + n*num - total for x in big}
                big.add(n*num - total)
            else:
                small |= {x + total - n*num for x in small}
                small.add(total - n*num)
            if big & small: # if big and small has common element
                return True
        return False

**idea** loop through all the possible length from 1 to n/2  
notes:
1. don't add one number multiple times
2. newDict[k+a] = oldDict[k] + 1
3. memorize all the possible length of a possible value since our length is fixed

In [3]:
class Solution(object):
    def splitArraySameAverage(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        n = len(A)
        avg, M = sum(A)/float(n), n/2
        for i in range(1,M+1):
            target = avg * i
            if target == int(target): 
                possible = {0:{0}}
                for a in A: 
                    d = {}
                    for k in possible.keys():
                        values = list(possible[k])
                        if k+a in possible:
                            d[k+a] = list(possible[k+a])
                        for v in values:    
                            if k+a not in d: d[k+a] = [v+1]
                            else: 
                                d[k+a].append(v+1)
                    for k in d:
                        possible[k] = set(d[k])
                    if target in possible and i in possible[target]: return True
        return False

## 3. 355 - Design Twitter
Design a simplified version of Twitter where users can post tweets, follow/unfollow another user and is able to see the 10 most recent tweets in the user's news feed. Your design should support the following methods:

**postTweet(userId, tweetId):** Compose a new tweet.  
**getNewsFeed(userId):** Retrieve the 10 most recent tweet ids in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user herself. Tweets must be ordered from most recent to least recent.  
**follow(followerId, followeeId):** Follower follows a followee.  
**unfollow(followerId, followeeId):** Follower unfollows a followee.

### set 
|: or  
discard: remove specific value

In [5]:
class Twitter(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.tweets = collections.defaultdict(list)
        self.followees = collections.defaultdict(set)
        self.order = 1
        

    def postTweet(self, userId, tweetId):
        """
        Compose a new tweet.
        :type userId: int
        :type tweetId: int
        :rtype: void
        """
        self.tweets[userId].append((self.order, tweetId))
        self.order += 1
        

    def getNewsFeed(self, userId):
        """
        Retrieve the 10 most recent tweet ids in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user herself. Tweets must be ordered from most recent to least recent.
        :type userId: int
        :rtype: List[int]
        """
        news = []
        for user in self.followees[userId] | {userId}:
            news += self.tweets[user]
        news.sort(reverse=True)
        return [x[1] for x in news[:min(len(news),10)]]
        

    def follow(self, followerId, followeeId):
        """
        Follower follows a followee. If the operation is invalid, it should be a no-op.
        :type followerId: int
        :type followeeId: int
        :rtype: void
        """
        self.followees[followerId] = self.followees.get(followerId,set()) | {followeeId}

    def unfollow(self, followerId, followeeId):
        """
        Follower unfollows a followee. If the operation is invalid, it should be a no-op.
        :type followerId: int
        :type followeeId: int
        :rtype: void
        """
        if followeeId in self.followees[followerId]:
            if len(self.followees[followerId]) == 1:
                self.followees[followerId] = set()
            else:
                self.followees[followerId].discard(followeeId)
        

# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)